In [1]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV 

import nltk 
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import pickle
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# Creating a list for columns to keep
cols = ['EventDescription','FailedAssets','IncidentCause','IncidentConsequence','IncidentType','WeatherStation','Status','Category']

In [3]:
# Importing file
df = pd.read_csv('cleaned_incidents1.csv', usecols=cols)
print(df.shape)

#dropping nulls
df = df.dropna()
df.isnull().sum()

(6504, 8)


EventDescription       0
FailedAssets           0
IncidentCause          0
IncidentConsequence    0
IncidentType           0
Status                 0
WeatherStation         0
Category               0
dtype: int64

In [4]:
df['Description'] = df['WeatherStation'] + ' ' + df['IncidentType'] + ' ' + df['Status'] + ' ' + df['EventDescription'] + ' ' + df['FailedAssets'] + ' ' + df['IncidentCause']+ ' ' + df['IncidentConsequence']

In [5]:
print(df['Description'])

0       Avalon Airport Infrastructure (network-based) ...
1       Laverton Raaf Infrastructure (network-based) R...
2       Warrnambool Airport Ndb Infrastructure (networ...
3       Essendon Airport Infrastructure (network-based...
4       Avalon Airport Infrastructure (network-based) ...
                              ...                        
6499    Moorabbin Airport Infrastructure (network-base...
6500    Avalon Airport Infrastructure (network-based) ...
6501    Mildura Airport Infrastructure (network-based)...
6502    Swan Hill Aerodrome Infrastructure (network-ba...
6503    Moorabbin Airport Infrastructure (network-base...
Name: Description, Length: 6488, dtype: object


Stopwords, Splitting, Label Encoding

In [6]:
# Creating stopwords list

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
numerical_symbols = re.compile('0-90-9a-z')
 
STOPWORDS = set(stopwords.words('english'))
 
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = numerical_symbols.sub('', text)
    text = text.replace('x', '')
    #text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

df['Description'] = df['Description'].apply(clean_text)


'''
def getLemmText(text):
    tokens=word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    
    #ps = PorterStemmer()
    #tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)
df['Description'] = list(map(getLemmText,df['Description']))


def getStemmText(text):
    tokens=word_tokenize(text)
    ps = PorterStemmer()
    tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)
df['Description'] = list(map(getStemmText,df['Description']))

'''

"\ndef getLemmText(text):\n    tokens=word_tokenize(text)\n    lemmatizer = WordNetLemmatizer()\n    tokens=[lemmatizer.lemmatize(word) for word in tokens]\n    \n    #ps = PorterStemmer()\n    #tokens=[ps.stem(word) for word in tokens]\n    return ' '.join(tokens)\ndf['Description'] = list(map(getLemmText,df['Description']))\n\n\ndef getStemmText(text):\n    tokens=word_tokenize(text)\n    ps = PorterStemmer()\n    tokens=[ps.stem(word) for word in tokens]\n    return ' '.join(tokens)\ndf['Description'] = list(map(getStemmText,df['Description']))\n\n"

In [7]:
print(df['Description'])

0       avalon airport infrastructure networkbased rep...
1       laverton raaf infrastructure networkbased repo...
2       warrnambool airport ndb infrastructure network...
3       essendon airport infrastructure networkbased r...
4       avalon airport infrastructure networkbased rep...
                              ...                        
6499    moorabbin airport infrastructure networkbased ...
6500    avalon airport infrastructure networkbased rep...
6501    mildura airport infrastructure networkbased re...
6502    swan hill aerodrome infrastructure networkbase...
6503    moorabbin airport infrastructure networkbased ...
Name: Description, Length: 6488, dtype: object


In [8]:
# Label encoding for Category
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'].astype(str))
#store the 'Category' variable in Y
X = df[['Description']]
Y = df[['Category']]

In [9]:
# Splitting of data in test and train
x_train, x_test, y_train, y_test = train_test_split(df['Description'],Y, 
                                                    test_size=0.25, random_state=4)

XGBoost

In [10]:
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
vectorizer = TfidfVectorizer(stop_words=stopwords, analyzer='word', max_features=1000)
tfidf = vectorizer.fit(df['Description'])
# tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

In [12]:
x_train_tfidf =  tfidf.transform(x_train)
x_test_tfidf =  tfidf.transform(x_test)

In [13]:
!pip install numpy Cython

!pip install pymrmr

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for pymrmr: filename=pymrmr-0.1.8-cp36-cp36m-linux_x86_64.whl size=256734 sha256=ac5d7007350717be229cbeae9582daf19e10a7a8434e15f3a597e9901d1ea350
  Stored in directory: /root/.cache/pip/wheels/5b/ce/3a/bc9b80047f68973d909a35bb8e3062b7c7377510607ec35998
Successfully built pymrmr


In [14]:
# df_mrmr = pd.DataFrame(x_train_tfidf)

In [15]:
# import pymrmr

# pymrmr.mRMR(df_mrmr, 'MID', 7)
# # print(df_mrmr)

In [16]:
# print(x_train_tfidf)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier(learning_rate=0.1, verbosity=2)
clf.fit(x_train_tfidf, y_train)

In [18]:
# Model evaluation
prediction = clf.predict(x_test_tfidf)

acc = accuracy_score(y_test, prediction).round(4)
print("Accuracy using TF-IDF is: {}%".format(acc * 100.0))

Accuracy using TF-IDF is: 81.81%


Hyper-parameter Tuning

In [75]:
# grid search
param_grid = {
    'learning_rate': [0.10, 0.20, 0.50],
    'max_depth' : [3, 4],
    'verbosity' : [1, 2],
    'min_child_weight' :[1, 2],
    # 'gamma' : [0.1, 0.2],
    # 'colsample_bytree' : [0.3, 0.4, 0.5, 0.7]
}
cv_xgb = GridSearchCV(estimator=XGBClassifier(random_state=4),
                      param_grid=param_grid, cv= 5, n_jobs=-1)
cv_xgb.fit(x_train_tfidf, y_train)
cv_xgb.best_params_
print(cv_xgb.best_score_)

[06:25:20] ======== Monitor: Learner ========
[06:25:20] ======== Monitor: GBTree ========


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.836826766219761


In [77]:
print(cv_xgb.best_params_)
#Best Parameters : {learning_rate : 0.2, max_depth: 4, min_child_weight: 1, verbosity: 1}

{'learning_rate': 0.2, 'max_depth': 4, 'min_child_weight': 1, 'verbosity': 1}


Optimized Model with Best Parameters

In [78]:
# clf = XGBClassifier(learning_rate=0.5, max_depth=3, gamma= 0.1, min_child_weight=5, verbosity=3)
clf = XGBClassifier(learning_rate=0.2, max_depth=4, verbosity=1, min_child_weight=1)
clf.fit(x_train_tfidf, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [79]:
prediction = clf.predict(x_test_tfidf)

acc = accuracy_score(y_test, prediction).round(4)
print("Accuracy using TF-IDF is: {}%".format(acc * 100.0))

Accuracy using TF-IDF is: 82.31%
